In [15]:
import cv2
import numpy as np
import os

numx = 0

def make_data():
    
	imageData=[]
	labels=[]
	data=[]
	features=28*28*3
	noImage=96000
	nClass=96
	path=os.getcwd()
	folderpath=os.path.join(path,"classes")
	vidpath=os.path.join(path,"vimage")

	label=-1
	for length in range(2):
		for width in range(2):
			for angle in range(12):
				for color in range(2):

					string=str(length)+"_"+str(width)+"_"+str(angle)+"_"+str(color)
					fpath=folderpath+"/"+string

					ipath=fpath+"/"+str(length)+"_"+str(width)+"_"+str(angle)+"_"+str(color)
					label=label+1

					for image in range(1000):
						iName=ipath+"_"+str(image)+".jpeg"
						img_data=cv2.imread(iName)
						data.append(img_data.flatten().reshape(features))
						labels.append(label)
	imageData=np.array(data,float).reshape(noImage,features)
	labels=np.array(labels,float).reshape(noImage,1)
	imageData=imageData/255.0
	data=np.concatenate((imageData,labels),axis=1)
	np.random.shuffle(data)
	data=data.reshape(noImage,features+1)


	ipt=data[:,:-1]
	opt=np.array(data[:,-1],int).reshape(noImage,1)
	b=np.zeros((noImage,nClass),int)
	for i in range(noImage):
		b[i,opt[i]]=1

	return ipt,b




In [16]:
x,y=make_data()
print(x.shape,y.shape)

(96000, 2352) (96000, 96)


In [17]:
nx = x
ny = y

In [51]:
x = nx
y = ny

In [52]:
accuracyList = []
epochList = []

In [ ]:
import tensorflow as tf

print(x.shape)
splitSize = int(x.shape[0]*0.7)
x, testData = x[:splitSize,:], x[splitSize:,:]
y, testY = y[:splitSize,:], y[splitSize:,:]

batchPtr = 0
# splitting the data into train, val,test
seed = 128
rng = np.random.RandomState(seed)

def batchForm():
    # form the batch of given batch size
    global batchPtr
    batchX = x[batchPtr:batchPtr+batchSize,:]
    batchY = y[batchPtr:batchPtr+batchSize,:]
    batchPtr += batchSize
#     print(batchX)
#     print(batchY)
    return batchX,batchY

noInputUnit = 784*3
noHiddenUnit = 10
noOutputUnit = 96
batchSize = 200

# placeholder to feed values during 
x1 = tf.placeholder(tf.float32, [None, noInputUnit])
y1 = tf.placeholder(tf.float32, [None, noOutputUnit])

# set remaining parameters
epochs = 100
learningRate = 0.01

hiddenWeight = tf.Variable(tf.random_normal([noInputUnit, noHiddenUnit], seed=seed))
outputWeight = tf.Variable(tf.random_normal([noHiddenUnit, noOutputUnit], seed=seed))

hiddenBias = tf.Variable(tf.random_normal([noHiddenUnit], seed=seed))
outputBias = tf.Variable(tf.random_normal([noOutputUnit], seed=seed))

hiddenLayer = tf.add(tf.matmul(x1, hiddenWeight), hiddenBias)
hiddenLayer = tf.nn.relu(hiddenLayer)

outputLyer = tf.matmul(hiddenLayer, outputWeight) + outputBias

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=outputLyer, labels=y1))

optimizer = tf.train.AdamOptimizer(learning_rate=learningRate).minimize(cost)

init = tf.initialize_all_variables()

for xx in range(1, 55):
    with tf.Session() as sess:
        # create initialized variables
        sess.run(init)        
        for epoch in range(epochs):
            avgCost = 0
            total_batch = xx
    #         print(totalBatch)
    #         print(x.shape[0])
            batchPtr = 0
            for i in range(totalBatch):
                batchX, batchY = batchForm()
                _, c = sess.run([optimizer, cost], feed_dict = {x1: batchX, y1: batchY})
                avgCost += c / totalBatch 
                if xx == 53:
                    epochList.append(avg_cost)

#             print("Epoch:", (epoch+1), "cost =", "{:.5f}".format(avgCost))

        pred_temp = tf.equal(tf.argmax(outputLyer, 1), tf.argmax(y1, 1))
        accuracy = tf.reduce_mean(tf.cast(pred_temp, "float"))
        accuracy = tf.reduce_mean(tf.cast(pred_temp, "float"))
        ac = accuracy.eval({x1: val_x.reshape(-1, 784*3), y1: val_y})
        print("Testing Accuracy:", ac)
        accuracyList.append(ac)

(96000, 2352)
Testing Accuracy: 0.9389236
Testing Accuracy: 0.9389236
Testing Accuracy: 0.9389236
Testing Accuracy: 0.9389236
Testing Accuracy: 0.9389236
Testing Accuracy: 0.9389236
Testing Accuracy: 0.9389236
Testing Accuracy: 0.9389236


In [ ]:
import matplotlib.pyplot as plt
def plot(values, nx, title, xx, yy):
    plt.figure()
    name = nx + "_curve.jpg"
    name = str(hidden_num_units)+"_"+str(epochs) + name
    name = "./graphs/my_" + name
    xVal = np.arange(len(values)) * 200
#     print(xVal)
#     xVal.append(32)
#     print(xVal)
    plt.plot(xVal, values, marker='o',markersize='4', linestyle='--')
    plt.ylabel(yy)
    plt.xlabel(xx)
    plt.title(title)
#     plt.show()
    plt.savefig(name)
    
plot(np.array(accuracyList), "", "Learning Curve", "Train Data Size", "Accuracy")
plot(np.array(epochList), "epoch", "Epoch vs Loss", "Epochs", "Loss Function" )